In [1]:
import duckdb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import shutil
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import numpy as np
import torch

/home/jadson/anaconda3/envs/py/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Conectar (ou criar) um banco de dados
con = duckdb.connect(database='jus.duckdb', read_only=False)

In [3]:
con.execute("""
CREATE TABLE metadadosPublicacao202202 AS SELECT * FROM read_json_auto('./data/202202-stj/metadadosPublicacao202202.json');
""")

In [4]:
# tabela de texto dos juiz
con.execute("""
  CREATE TABLE decisions_description (
    ID INT PRIMARY KEY,
    content TEXT,
  );
""")

In [ ]:
# tabela de texto dos juiz
con.execute("""
  ALTER TABLE decisions_description ADD CONSTRAINT fk_decisions
  FOREIGN KEY (doc_id) REFERENCES decisions(seqDocumento);
""")

In [4]:
tokenizer_trad = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-mul-en",top_k=124)
m_trad = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-mul-en",top_k=124)

In [ ]:
import argostranslate.package
import argostranslate.translate


In [ ]:
from_code = "pt"
to_code = "en"

# Download and install Argos Translate package
argostranslate.package.update_package_index()
available_packages = argostranslate.package.get_available_packages()
package_to_install = next(
    filter(
        lambda x: x.from_code == from_code and x.to_code == to_code, available_packages
    )
)
argostranslate.package.install_from_path(package_to_install.download())

# Translate
translatedText = argostranslate.translate.translate("Hello World", from_code, to_code)
print(translatedText)
# '¡Hola Mundo!'

In [5]:
translated = m_trad.generate(**tokenizer_trad("PROCESSUAL CIVIL. OFENSA AO ART. 1.022 DO CPC/2015 NÃO CONFIGURADA. EXECUÇÃO FISCAL. MULTA. INFRAÇÃO ADMINISTRATIVA. MASSA FALIDA. INEXIGIBILIDADE.", return_tensors="pt", add_special_tokens=True))

for t in translated:
    print(tokenizer_trad.decode(t, skip_special_tokens=True) )

CIVIL PROCESSING TO ARTICLE 1.0022 OF CPC/2015 NOT CONFIGUARDED. FISCAL EXECUTION. MULTA. ADMINISTRATIVE INFRATION. MASS FAILED. INEXIGIBILITY.


In [10]:
def create_segments(text, tokenizer):
    # Split text into sentences
    sentences = text.splitlines()

    segments = []
    current_segment = np.array([], dtype=int)
    current_token_count = 0

    for sentence in sentences:
        tokens = tokenizer(sentence, return_tensors="pt", add_special_tokens=True).input_ids[0].numpy()
        token_count = tokens.shape[0]

        # If the sentence itself is larger than 512 tokens, we need to handle it separately
        if token_count >= 512:
            # Save current segment if not empty
            if current_segment.size > 0:
                segments.append(current_segment)
                current_segment = np.array([], dtype=int)
                current_token_count = 0

            # Split the large sentence and add to segments
            for i in range(0, token_count, 512):
                segments.append(tokens[i:i+512])
            continue

        # Add tokens to current segment or create new segment
        if current_token_count + token_count < 512:
            current_segment = np.concatenate((current_segment, tokens))
            current_token_count += token_count
        else:
            segments.append(current_segment)
            current_segment = tokens
            current_token_count = token_count

    if current_segment.size:
        segments.append(current_segment)

    result = []
    # Preenchendo segmentos com zeros para que tenham tamanho 512
    for segment in segments:
        if segment.size > 0:
            padding_length = 512 - segment.shape[0]
            result.append(np.concatenate([segment, np.zeros(padding_length, dtype=int)]))

    return result



In [11]:
segments = create_segments("Quais são os benefícios de banana ?",tokenizer_trad)

In [12]:
for segment in segments:
    translated = m_trad.generate(torch.tensor([segment]))
    print(translated)
    for t in translated:
        print(tokenizer_trad.decode(t, skip_special_tokens=True) )


/tmp/ipykernel_861129/511425395.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343904639/work/torch/csrc/utils/tensor_new.cpp:245.)
  translated = m_trad.generate(torch.tensor([segment]))


tensor([[64171,    21,    15,    41,  5394,     2,     0]])
I'm sorry.


In [14]:
with open("./data/202202-stj/20220201/94552609.txt", 'r') as f:
  content = f.read()
  segments = create_segments(content,tokenizer_trad)
  
  for i,segment in enumerate(segments):
      print(i,segment.size)
      translated = m_trad.generate(torch.tensor([segment]))
      print(translated)
      for t in translated:
          print(tokenizer_trad.decode(t, skip_special_tokens=True))

0 512
tensor([[64171,     4,     2,     4,     2,     4,     2,     4,     2,     4,
             2,     4,     2,     4,     2,     4,     2,     4,     2,     4,
             2,     4,     2,     4,     2,     4,     2,     4,     2,     4,
             2,     4,     2,     4,     2,     4,     2,     4,     2,     4,
             2,     4,     2,     4,     2,     4,     2,     4,     2,     4,
             2,     4,     2,     4,     2,     4,     2,     4,     2,     4,
             2,     4,     2,     4,     2,     4,     2,     4,     2,     4,
             2,     4,     2,     4,     2,     4,     2,     4,     2,     4,
             2,     4,     2,     4,     2,     4,     2,     4,     2,     4,
             2,     4,     2,     4,     2,     4,     2,     4,     2,     4,
             2,     4,     2,     4,     2,     4,     2,     4,     2,     4,
             2,     4,     2,     4,     2,     4,     2,     4,     2,     4,
             2,     4,     2,     4,     2,   

KeyboardInterrupt: 

In [23]:
def split_encoded_text(encoded_text, max_length=512):
    total_length = encoded_text['input_ids'].shape[1]
    
    chunks = []
    for i in range(0, total_length, max_length):
        end = min(i+max_length, total_length)
        chunk = {
            "input_ids": encoded_text["input_ids"][0, i:end].unsqueeze(0),
            "attention_mask": encoded_text["attention_mask"][0, i:end].unsqueeze(0)
        }
        chunks.append(chunk)
    return chunks

In [39]:
"""
EMENTA
PROCESSUAL CIVIL. OFENSA AO ART. 1.022 DO CPC/2015 NÃO CONFIGURADA. EXECUÇÃO FISCAL. MULTA. INFRAÇÃO ADMINISTRATIVA. MASSA FALIDA. INEXIGIBILIDADE. DECRETO-LEI 7.661/1945. FALÊNCIA DECRETADA ANTES DA VIGÊNCIA DA LEI 11.101/2005. HONORÁRIOS ADVOCATÍCIOS. ART. 20, § 4º, DO CPC/1973. MINORAÇÃO DA QUANTIA ARBITRADA NA CORTE REGIONAL. REVISÃO DAS PREMISSAS FÁTICAS E PROBATÓRIAS. SÚMULA 7/STJ.
1. A solução integral da controvérsia, com fundamento suficiente, não caracteriza ofensa ao art. 1.022 do CPC/2015.
2. A cobrança da multa administrativa na via da Execução Fiscal, com base nos arts. 2º, 29 e 39, § 2º, da Lei 6.830/1980 no caso dos autos não foi debatida na instância de origem. Desse modo, carece o tema do prequestionamento viabilizador do Recurso Especial, razão pela qual não merece ser apreciado.
ACÓRDÃO RECORRIDO
3. O acórdão recorrido consignou: "A sentença merece reparo. Trata-se de apelações contra sentença que julgou procedentes os embargos interpostos à execução fiscal nº: 0113746-80.2014.4.02.5101, que, por sua vez visa executar dívida originada de multa administrativa e multa monitória aplicadas pela Comissão de Valores Mobili
"""
with open("./data/202202-stj/20220201/94552609.txt", 'r') as f:
  content = f.read()
  encoded_text = tokenizer_trad(content, return_tensors="pt", add_special_tokens=True)

  chunks = split_encoded_text(encoded_text)
  #print(chunks)

  for i,segment in enumerate(chunks):
      print(i,segment["input_ids"][0].shape)
      print(tokenizer_trad.decode(segment["input_ids"][0], skip_special_tokens=True))
      #frase = ""
      #for t in segment["input_ids"][0]:
      #    frase = frase + tokenizer_trad.decode(t, skip_special_tokens=True)
      #print(frase)

      translated = m_trad.generate(**segment)
      print(translated)
      for t in translated:
          print(tokenizer_trad.decode(t, skip_special_tokens=True))

0 torch.Size([512])
EMENTA PROCESSUAL CIVIL. OFENSA AO ART. 1.022 DO CPC/2015 NÃO CONFIGURADA. EXECUÇÃO FISCAL. MULTA. INFRAÇÃO ADMINISTRATIVA. MASSA FALIDA. INEXIGIBILIDADE. DECRETO-LEI 7.661/1945. FALÊNCIA DECRETADA ANTES DA VIGÊNCIA DA LEI 11.101/2005. HONORÁRIOS ADVOCATÍCIOS. ART. 20, § 4o, DO CPC/1973. MINORAÇÃO DA QUANTIA ARBITRADA NA CORTE REGIONAL. REVISÃO DAS PREMISSAS FÁTICAS E PROBATÓRIAS. SÚMULA 7/STJ. 1. A solução▁integral da controvérsia, com fundamento suficiente,▁não▁caracteriza ofensa▁ao art. 1.022 do CPC/2015. 2. A cobrança da▁multa▁administrativa na via da Execução Fiscal, com base▁nos arts. 2o, 29 e 39, § 2o, da Lei 6.830/1980 no▁caso▁dos autos▁não▁foi debatida na instância de▁origem. Desse modo, carece o tema do prequestionamento viabilizador do Recurso Especial,▁razão pela▁qual▁não merece ser▁apreciado. ACÓRDÃO RECORRIDO 3. O▁acórdão recorrido consignou: "A sentença merece reparo. Trata-se de apelações contra sentença▁que julgou▁procedentes▁os embargos interpostos

In [89]:
# Caminho da pasta contendo os arquivos de texto
folder_path = './data/202202-stj/'

# Percorre a pasta e suas subpastas de forma recursiva
for subdir, dirs, files in os.walk(folder_path):
    for filename in files:
        if filename.endswith('.txt'):
            file_path = os.path.join(subdir, filename)
            # Extrai o ID do nome do arquivo
            file_id = int(filename.split('.')[0])
            with open(file_path, 'r') as f:
                content = f.read()
                segments = create_segments(content)
                print(segments[0].size)
                break
                for segment in segments:
                    translated = m_trad.generate(torch.tensor([segment]))
                    print(translated)
                    for t in translated:
                        print(tokenizer_trad.decode(t, skip_special_tokens=True) )

                #con.execute("INSERT INTO decisions_description (id,content) VALUES (?,?)", (file_id,content))
                print(file_id)
                break

403
4787
1927
3023
4748
5951
8608
369
2840
469
12499
419
4125
3975
3338
507
456
2939
346


In [ ]:
con.execute("""
DELETE FROM metadadosPublicacao202202 WHERE teor NOT IN ('Concedendo', 'Negando') or teor IS NULL;
""")

In [ ]:
data = con.execute("SELECT seqDocumento,teor FROM metadadosPublicacao202202").df()
data

In [ ]:
# Drop rows with missing values and reset the index
data = data.dropna().reset_index(drop=True)
#35404 rows × 3 columns

In [ ]:
teor_type = LabelEncoder()
data['teor_encoder'] = teor_type.fit_transform(data['teor'])
data

In [ ]:
# Para saber a correspondência entre o valor codificado e o rótulo original:
for index, label in enumerate(teor_type.classes_):
    print(f"{index}: {label}")

In [ ]:
# Para saber a correspondência entre o valor codificado e o rótulo original:
for index, label in enumerate(teor_type.classes_):
    print(f"{index}: {label}")
    con.execute("""
    UPDATE metadadosPublicacao202202
    SET teor = ?
    WHERE teor = ?;
    """,(index,label))

In [ ]:
data = con.execute("SELECT seqDocumento,teor FROM metadadosPublicacao202202").df()
data

In [ ]:
data.dtypes